# Assignment 2 Workbook

## Question 1a: SVM Primal Form

Please implement the training and testing algorithms of soft-margin Linear Support Vector Machine in its primal form, that is,

$$\min_{\mathbf{w},b,\{\xi_i\}} \frac{1}{2} \|\mathbf{w}\|_2^2 + C\sum_{i=1}^N \xi_i \nonumber \\ s.t.~~ y_i (\mathbf{w}^\top \mathbf{x}_i + b) \ge 1 - \xi_i, ~~\forall i \nonumber \\ \xi_i \ge 0 \nonumber$$
Use CVXPY in your implementation strictly following the format given in this Notebook, and supplying missing code **in the indicated space. Do not add or change any other code**


In [ ]:
# import required libraries
!pip install cvxpy --upgrade # if needed
import cvxpy as cp
import pandas as pd
import numpy as np

In [2]:
# get training dataset
train = "train1.csv"
df = pd.read_csv(train, header=None)
X_train = df[:1500].iloc[:, 1:].to_numpy()
Y_train = df[:1500].iloc[:, 0].replace(0, -1).to_numpy()

In [3]:
# get test dataset
test = "test1.csv"
df = pd.read_csv(test, header=None)
X_test = df.iloc[:500, 1:].to_numpy()
Y_test = df.iloc[:500, 0].replace(0, -1).to_numpy()

In [4]:
# train linear svm in primal form
def svm_train_primal(data_train, label_train, C):
    X, Y = data_train, label_train
    n_samples, m_features = np.shape(X)
    
    W_value = 0
    b_value = 0
    slack_var_value = 0

    W = cp.Variable(m_features)
    b = cp.Variable()
    slack_var = cp.Variable(n_samples)
    
    objective = cp.Minimize(0.5 * cp.sum_squares(W) + C * cp.sum(slack_var))
    constraints = [cp.multiply(Y, (X @ W + b)) >= 1 - slack_var, slack_var >= 0]
    
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    W_value = W.value
    b_value = b.value
    slack_var_value = slack_var.value

    return [W_value, b_value, slack_var_value]

# train primal model
C = 1
model_primal = svm_train_primal(X_train, Y_train, C)

# output svm primal form solutions
W = model_primal[0]
b = model_primal[1]
slack = model_primal[2]

print('Sum of W, b and slack:')
print(np.round(np.sum(W)+np.sum(slack)+b,2))


Sum of W, b and slack:
0.46


## Question 1b: SVM Primal Accuracy

Please complete and run this code and copy the result into Assignment 2 Question 1b

In [5]:
# predict accuracy of svm model on test dataset
def svm_predict(data_test, label_test, svm_model):
    
    acc = 0.0
    
    W_value, b_value, slack_var_value = svm_model
    
    predictions = np.sign(np.dot(data_test, W_value) + b_value)
    correct_predictions = np.sum(predictions == label_test)
    acc = correct_predictions / len(label_test)

    return acc

# predict and output primal accuracy
accuracy = svm_predict(X_test, Y_test, model_primal)
print('Accuracy:')
print(np.round(accuracy,2))

Accuracy:
0.94


## Question 2a: SVM Dual Form

Please implement the training and testing algorithms of soft-margin Linear Support Vector Machine in its **dual** form, that is,

$$\max_{\alpha_i}\sum_i \alpha_i - \frac{1}{2}\sum_i \sum_j \alpha_i \alpha_j y_i y_j <\mathbf{x}_i, \mathbf{x}_j> \nonumber \\ s.t. ~~~ 0 \le \alpha_i \le C\nonumber \\ ~~~ \sum_i \alpha_i y_i = 0 \nonumber$$

Use CVXPY in your implementation strictly following the format given in this Notebook, and supplying missing code in the indicated space. **Do not modify any other code**.


In [ ]:
# import required libraries
import cvxpy as cp
import pandas as pd
import numpy as np

In [ ]:
# get training dataset
train = "train1.csv"
df = pd.read_csv(train, header=None)
X_train = df[:1500].iloc[:, 1:].to_numpy()
Y_train = df[:1500].iloc[:, 0].replace(0, -1).to_numpy()

In [ ]:
# get test dataset
test = "test1.csv"
df = pd.read_csv(test, header=None)
X_test = df.iloc[:500, 1:].to_numpy()
Y_test = df.iloc[:500, 0].replace(0, -1).to_numpy()

In [ ]:
# train linear svm in dual form
def svm_train_dual(data_train, label_train, C):
    
    alphas = 0
    x, y = data_train, label_train
    n_samples, n_features = np.shape(x)
    

    K = np.dot(x, x.T)  # Compute the kernel (linear kernel)
    alpha = cp.Variable(n_samples)
    
    # Ensure the kernel matrix is symmetric
    P = cp.multiply(y[:, None] * y[None, :], K)
    P_sym = 0.5 * (P + P.T)  # Make sure P is symmetric
    
    objective = cp.Maximize(cp.sum(alpha) - 0.5 * cp.quad_form(alpha, P_sym))
    constraints = [alpha >= 0, alpha <= C, cp.sum(cp.multiply(alpha, y)) == 0]
    
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.SCS)
    
    alphas = alpha.value


    # return svm dual model alphas
    return alphas

# train dual model
c = 1
alphas = svm_train_dual(X_train, Y_train, c)

# output svm dual form solutions
print('Sum of alphas:')
print(np.round(np.sum(alphas),2))

## Question 2b: Dual model parameters

Complete the code where indicated, run it and copy the result into Assignment 2 Question 2b.

In [ ]:
# obtain primal w*, b* from dual solution
def find_model_params_from_dual(data, label, alphas, C):
    
    # this value is used to compare values generated by CVXPYY to zero.
    # for example, instead of 
    # if var > 0, we use: if var > zero_threshold
    zero_threshold = 0.0001
    n_samples, n_features = np.shape(data)
    a = alphas
   
    a[np.isclose(a, 0, atol=zero_threshold)] = 0  # zero out nearly zeros
    a[np.isclose(a, C, atol=zero_threshold)] = C  # round the ones that are nearly C


    # b is scalar, but may be sliglty diffrent for each Y, calculate b for each label and return mean of them
# ====================== YOUR CODE HERE ====================== 
    w_dual = np.sum(a[:, None] * label[:, None] * data, axis=0)

    # Find support vectors (indices where 0 < a < C)
    support_vector_indices = np.where((a > zero_threshold) & (a < C))[0]

    b_dual_values = []
    for i in support_vector_indices:
        b_dual_i = label[i] - np.dot(w_dual, data[i])
        b_dual_values.append(b_dual_i)

    b_dual = np.mean(b_dual_values)
 
# DO NOT use any other import statements for this question
# Note: b_dual is a scalar, but maybe different for each label, therefore calculate the mean


# =========================================================================
    return w_dual, b_dual

# output reconstructed w* and b* from svm dual problem
C = 1
model_dual = find_model_params_from_dual(X_train, Y_train, alphas, C)

print('Sum of W and b:')
print(np.round(np.sum(model_dual[0])+model_dual[1],2))

## Question 3a: PCA implementation

1. Perform PCA on the dataset to reduce each sample into a 10-dimensional feature vector.
2. Print the required result and enter into Assignment 2 Question 3a.

=========================================================================
- Implementing PCA algorithm.
    - Start
        - Input: number of samples as matrix $X$ of $m$ rows and $k$ columns.
        - Calculate the mean for each column. $$mean = \frac {1}{m} \sum \limits _{i=1} ^{n}X_{ij}$$
        - Calculate the centralised matrix $X_C$ and covariance matrix $C$. $$X_C=X-mean$$ $$C = \frac {1}{m}(X_C)^TX_C$$
        - Calculate the eigenvalues and eigenvectors using convariance matrix.
        - Select top x principal components - which are eigen vector corresponding to top x eigen values. Construct matrix $P$.
    - End
    
- Transforming the the data using the principal components (matrix $P$) obtained using the PCA algorithm. $$Transformed \: Data = XP$$
- Calculating the covariance matrix of the transformed data by first centralising it(mean subtracted) and then obtaining the covariance matrix.

In [16]:
# import required libraries
import pandas as pd
import numpy as np

In [17]:
# get training dataset
train = "train1.csv"
df = pd.read_csv(train, header=None)
X = df[:1500].iloc[:, 1:].to_numpy()

In [18]:
# Selecting top 10 Principal components
no_of_components = 10

covariance_matrix_X = 0
covariance_matrix_X_transformed = 0

# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question
# Centralize the data
X_mean = np.mean(X, axis=0)
X_centered = X - X_mean

# Compute the covariance matrix
covariance_matrix_X = np.cov(X_centered, rowvar=False)

# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix_X)

# Sort eigenvalues and corresponding eigenvectors in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

# Select the top 10 eigenvectors (principal components)
P = sorted_eigenvectors[:, :no_of_components]

# Transform the data using the principal components
X_transformed = np.dot(X_centered, P)

# Compute the covariance matrix of the transformed data
X_transformed_centered = X_transformed - np.mean(X_transformed, axis=0)
covariance_matrix_X_transformed = np.cov(X_transformed_centered, rowvar=False)



# ==========================================================================

sum_cov_X = np.sum(covariance_matrix_X)
sum_cov_X_transformed = np.sum(covariance_matrix_X_transformed)

print("sum_cov_X_transformed:")
print(np.round(sum_cov_X + sum_cov_X_transformed,2))

sum_cov_X_transformed:
314.96


## Question 3b: PCA data reconstructon

For this question:
1. Reconstruct the X dataset from your results in code Question 3a as X_back.
1. Calculate covariance matrix for X_back and enter its sum into Assignment 2 Question 3b.

For this part, use the libraries imported in Question 3a, and do not import any moore libraries.

In [19]:
# Do not import any additional libraries for this section
# Do not change any code outside of this area marked with =============================

# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question
# Reconstruct the original dataset from the transformed data
X_back = np.dot(X_transformed, P.T) + X_mean

# Compute the covariance matrix of the reconstructed data
X_back_centered = X_back - np.mean(X_back, axis=0)
covariance_matrix_X_back = np.cov(X_back_centered, rowvar=False)


# ============================================================
print("Sum of covariance_matrix_X_back:")
print(np.round(np.sum(covariance_matrix_X_back),2))

Sum of covariance_matrix_X_back:
58.4


## Questions 4a and 4b: Kernel k-Means derivation

Complete these questions in myUni assignment 2

## Question 4c: Kernel k-Means implemention


In Question 4a and 4b you have proved the relationship between the variance of distances $$m_1 = \frac{1}{m} \sum_i\|\mathbf{x}_i - \mathbf{\mu}\|_2^2$$ and average of squared pairwise distances $$m_2= \frac{1}{m^2}\sum_i\sum_j \|\mathbf{x}_i - \mathbf{x}_j\|_2^2$$

In this implementation, you will use this relationship.

In this question you will implement kernel k-means with modified RBF-kernel, using SpectralClustering from sklearn to reduce the programming efford of the full implementation. 
RBF-kernel is as follows:

$$(\mathbf{x}_i,\mathbf{x}_j) = \exp(\frac{-\|\mathbf{x}_i - \mathbf{x}_j\|_2^2}{2m_1})$$

The modification of RBF kernel will be that the $m_1$ will be replaced with equivalent form of $m_2$ as you proved in Question 4a and selected in 4b. For example, if you proved that $m_1^2=m_2$, then you will implement

$$(\mathbf{x}_i,\mathbf{x}_j) = \exp(\frac{-\|\mathbf{x}_i - \mathbf{x}_j\|_2^2}{2\sqrt{m_2}})$$


For reference, please write the formula of your RBF kernel as
$$(\mathbf{x}_i,\mathbf{x}_j)=...your formula...$$


In order to calculate pairwise distance matrix, please use scipy.spatial distance function, which is alread pre-loaded in the code. <br>
Please check the documentation of the SpectralClustering function on how to use precomputed affinity matrix, which you are going to supply with the above specification.


In [20]:
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering
from scipy.spatial import distance

df = pd.read_csv('train1.csv', header = None)
X = df.iloc[:1000,1:].to_numpy(copy=True)
Y = df.iloc[:1000,:1].to_numpy(copy=True)

In [21]:
n_clusters = 10
centroids_rbf = np.array(n_clusters)

centroids_rbf = []

# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question
"""
# use random_state=0 in the SpectralClustering function to make the results reproducible.
"""
# Calculate the pairwise distance matrix
pairwise_dists = distance.cdist(X, X, 'sqeuclidean')

# Calculate m2 (average of squared pairwise distances)
m2 = np.mean(pairwise_dists)

# Define the modified RBF kernel
gamma = 1 / (2 * np.sqrt(m2))
rbf_kernel = np.exp(-gamma * pairwise_dists)

# Perform Spectral Clustering using the precomputed RBF kernel as affinity matrix
clustering = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=0)
clustering.fit(rbf_kernel)

# Find the centroids of the clusters
labels = clustering.labels_
centroids_rbf = np.array([X[labels == i].mean(axis=0) for i in range(n_clusters)])



# =================================================================
    
print('Sum of centroids:')
print(np.round(np.sum(centroids_rbf),2))


Sum of centroids:
17.32


## Question 5a: Adaboost questions

Answer questions in myUni assignment 2

## Question 5b: Adaboost code correction
The following code shows an incomplete implementation of Adaboost algorithm and contains some errors. The code does not implement the logic specific to Adaboost. Please modify it to make a correct implementation. After completing the implementation, run the code and copy the result into space provided in myUni Question 6.

**You can change only sections of the code as indicated.** <br>
**Please clearly comment the purpose of the code that you added or changed.**

In [22]:
# Do not change this section

# import required libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from math import exp, log as ln

# DO NOT use any other import statements for this question

In [23]:
# Do not change this section

# get training dataset
train = "train1.csv"
df = pd.read_csv(train, header=None)
X_train = df.iloc[:, 1:].to_numpy()
Y_train = df.iloc[:, 0].replace(0, -1).to_numpy()

In [24]:
# Do not change this section

# get test dataset
test = "test1.csv"
df = pd.read_csv(test, header=None)
X_test = df.iloc[:, 1:].to_numpy()
Y_test = df.iloc[:, 0].replace(0, -1).to_numpy()

In [25]:
# ================= Do not change this section
# DO NOT use any other import statements for this question

# sample_weights for training data
def weak_classifier_train(train_data, train_label, sample_weights):
    # Create a decision stump
    stump = DecisionTreeClassifier(max_depth=1)
    
    # Train the stump using the weighted samples
    stump.fit(train_data, train_label, sample_weight=sample_weights)
    
    # Generate predictions and calculate error
    model_prediction = stump.predict(train_data)
    train_label = np.array(train_label)
    model_prediction = np.array(model_prediction)
    sample_weights = np.array(sample_weights)

    # The following line should work if both train_label and model_prediction are numpy arrays of the same size.
    error = np.sum(sample_weights[train_label != model_prediction])
    
    return stump, error, model_prediction

def weak_classifier_prediction(test_data, model):
    
    # The model_param_t is our trained stump, so we use it to make predictions
    return model.predict([test_data])

You can add or change the code in the following section **only where indicated.** <br>
**Please clearly comment the purpose of the code that you added or changed.**

In [ ]:
# DO NOT use any other import statements for this question

def Adaboost_train(train_data, train_label, T):
    # train_data: N x d matrix
    # train_label: N x 1 vector
    # T: the number of weak classifiers in the ensemble

    ensemble_models = []
    alphas = []  # List of all alpha values

    N = len(train_label)  # The number of samples

    # Initialize weight distribution for each sample to be evenly distributed
    D = np.array([1 / N] * N)

    # For each weak classifier, do the following
    for t in range(T):

        # ====================== Add and/or correct code in this section ======================  
        
        # weak_classifier_train takes the training data and labels as well as the sample weight distribution
        # returns the model parameters and the error
        # model returns the model parameters of the learned weak classifier
        model = weak_classifier_train(train_data, train_label, D)  

 
        # ===============================================================================  

        # definition of model
        ensemble_models.append(model)

    return ensemble_models, alphas


def Adaboost_test(test_data, ensemble_models, alphas):
    # test_data: n x d
    predictions = []

    for i in range(len(test_data)):
        decision_ensemble = 0

        # ====================== Change or correct code in this section ======================  
 
        # For each model in the ensemble models, do the following
        for k in range(len(ensemble_models)):

            # Make a prediction on the classification of the sample
            prediction = weak_classifier_prediction(test_data[i], ensemble_models[k])  # prediction returns 1 or -1 prediction from the weak classifier

            # Add this classification multiplied by its confidence (alpha) to the sum (either +1 or -1)
            decision_ensemble += prediction

        # If more models predicted it as +1 class
        if decision_ensemble > 0:
            prediction = 1
        # Else, more models predicted it as -1 class
        else:
            prediction = -1
        
        # ============================================================================  
 
        predictions.append(prediction)

    return predictions

# predict and output accuracy

ensemble_models, alphas = Adaboost_train(X_train, Y_train, 3)
predicted_labels = Adaboost_test(X_test, ensemble_models, alphas)


In [26]:
# Adaboost training function
def Adaboost_train(train_data, train_label, T):
    # train_data: N x d matrix
    # train_label: N x 1 vector
    # T: the number of weak classifiers in the ensemble

    ensemble_models = []
    alphas = []  # List of all alpha values

    N = len(train_label)  # The number of samples

    # Initialize weight distribution for each sample to be evenly distributed
    D = np.array([1 / N] * N)

    # For each weak classifier, do the following
    for t in range(T):

        # Train weak classifier
        stump, error, model_prediction = weak_classifier_train(train_data, train_label, D)  

        # Compute alpha (model weight)
        alpha = 0.5 * ln((1 - error) / error)

        # Store the model and alpha
        ensemble_models.append(stump)
        alphas.append(alpha)

        # Update sample weights
        D = D * np.exp(-alpha * train_label * model_prediction)
        D = D / np.sum(D)  # Normalize to make it a distribution

    return ensemble_models, alphas

# Adaboost test function
def Adaboost_test(test_data, ensemble_models, alphas):
    # test_data: n x d
    predictions = []

    for i in range(len(test_data)):
        decision_ensemble = 0

        # For each model in the ensemble models, do the following
        for k in range(len(ensemble_models)):

            # Make a prediction on the classification of the sample
            prediction = ensemble_models[k].predict([test_data[i]])[0]  # prediction returns 1 or -1 prediction from the weak classifier

            # Add this classification multiplied by its confidence (alpha) to the sum
            decision_ensemble += alphas[k] * prediction

        # If the sum of weighted predictions is positive, predict +1, otherwise predict -1
        if decision_ensemble > 0:
            prediction = 1
        else:
            prediction = -1
        
        predictions.append(prediction)

    return predictions

# predict and output accuracy

ensemble_models, alphas = Adaboost_train(X_train, Y_train, 3)
predicted_labels = Adaboost_test(X_test, ensemble_models, alphas)
accuracy = accuracy_score(Y_test, predicted_labels)
print('Accuracy:', np.round(accuracy, 2))


Accuracy: 0.98


In [27]:
######## do not change this code ############

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

print('Accuracy:')
print(np.round(accuracy,3))

Accuracy:
0.983
